In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../data'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor
from smote import smote
from train import train_smote, test_smote

# Set device to GPU if available, else use CPU
device = torch.device("cuda")
print(f"Current device: {torch.cuda.get_device_name(torch.cuda.current_device())}" if torch.cuda.is_available() else "Current device: CPU")
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
args = Args() 
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../data.pt')
print(data)

HeteroData(
  a={
    num_nodes=28645,
    y=[28645]
  },
  p={ num_nodes=21044 },
  v={ num_nodes=18 },
  p_title_embed={ x=[21044, 128] },
  p_abstract_embed={ x=[21044, 128] },
  p_net_embed={ x=[21044, 128] },
  p_a_net_embed={ x=[21044, 128] },
  p_p_net_embed={ x=[21044, 128] },
  p_v_net_embed={ x=[21044, 128] },
  a_net_embed={ x=[28645, 128] },
  a_text_embed={ x=[28645, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 198912] },
  (a, walk, p)={ edge_index=[2, 144965] },
  (a, walk, v)={ edge_index=[2, 42500] },
  (p, walk, a)={ edge_index=[2, 137499] },
  (p, walk, p)={ edge_index=[2, 158408] },
  (p, walk, v)={ edge_index=[2, 29341] },
  (v, walk, a)={ edge_index=[2, 180] },
  (v, walk, p)={ edge_index=[2, 180] },
  (v, walk, v)={ edge_index=[2, 31] }
)


In [3]:
print(data['a','walk','p'].edge_index)

tensor([[    0,     0,     0,  ..., 28644, 28644, 28644],
        [11846, 11765, 12754,  ...,  4378, 20250,  6612]])


In [4]:
lr = 0.0001
num_epochs = 100
weight_decay = 5e-4
embed_dim = 128
dropout = 0.1
im_class_num = [1]
im_ratio = [0.4]
class_sample_num = 200
nclass = 4
node_dim = [args.A_n, args.P_n, args.V_n]

In [5]:
c_train_num = dl.train_num(data['a'].y, im_class_num, class_sample_num, im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[200, 80, 200, 200] 680
0 3508
1 1946
2 2890
3 3016
train_idx:  [27663, 23985, 9967, 13711, 8394, 12967, 21567, 12336, 3124, 5477, 16781, 27254, 17934, 7524, 20745, 10384, 3267, 13786, 8818, 27387, 16561, 5785, 15274, 23631, 16988, 28051, 4645, 16419, 12213, 16607, 2028, 13377, 12955, 2106, 16382, 968, 21052, 2975, 20841, 23222, 5333, 2499, 27712, 24960, 10459, 6646, 12605, 21265, 25358, 8146, 2054, 1379, 18334, 19603, 22384, 26060, 11601, 3548, 2989, 22012, 26653, 5330, 10996, 17760, 8425, 3904, 19583, 16043, 22263, 18261, 27107, 4890, 9512, 12699, 4310, 12711, 10569, 15854, 21015, 22665, 9592, 12831, 24283, 2737, 5924, 16681, 26909, 11155, 2497, 17431, 28408, 3201, 19838, 4421, 160, 35, 18142, 1526, 12519, 25028, 15543, 20774, 5787, 552, 2622, 12132, 15065, 9048, 7215, 21881, 4345, 12086, 4662, 20820, 20586, 23844, 23505, 4655, 15563, 5073, 22208, 26180, 20050, 16789, 7488, 16763, 3408, 181, 20018, 246, 9916, 8349, 1132, 38, 8039, 14420, 19133, 1611, 1521, 17871, 21853, 16895, 10717,

In [6]:
encoder = Het_En(node_dim, embed_dim, dropout)
decoder = EdgePredictor(embed_dim)
classifier = Classifier(embed_dim, nclass, dropout)
#print(features.shape)

In [7]:
# new_features, new_labels , new_train_idx = smote(features = data['a_net_embed'].x, labels = data['a'].y, 
#                                             train_idx = train_idx, portion = 0, im_class_num = im_class_num)
# print(new_features.shape, new_labels.shape, new_train_idx.shape)

## Training Part

In [8]:
torch.cuda.empty_cache()
train_smote(data, encoder, classifier, decoder, num_epochs, lr, weight_decay, train_idx, val_idx, portion = 0, im_class_num = im_class_num, mode = 'nsm')

Epoch [1/100], Loss: 1.4159, Accuracy: 0.1176, Edge Accuracy: 1.0000
Class 0:AUC-ROC- 0.5409, F1 Score- 0.0000; Class 1:AUC-ROC- 0.5837, F1 Score- 0.2105; Class 2:AUC-ROC- 0.4458, F1 Score- 0.0000; Class 3:AUC-ROC- 0.5280, F1 Score- 0.0773; Macro-Average AUC-ROC: 0.5246,Macro-Average F1 Score: 0.0719
Validation Loss: 1.3856, Validation Accuracy: 0.2500, Validation Edge Accuracy: 1.0000
Class 0:AUC-ROC- 0.7547, F1 Score- 0.0000; Class 1:AUC-ROC- 0.8084, F1 Score- 0.1272; Class 2:AUC-ROC- 0.4070, F1 Score- 0.0000; Class 3:AUC-ROC- 0.6214, F1 Score- 0.0150; Macro-Average AUC-ROC: 0.6479,Macro-Average F1 Score: 0.0356
Epoch [2/100], Loss: 1.4119, Accuracy: 0.1176, Edge Accuracy: 1.0000
Class 0:AUC-ROC- 0.6852, F1 Score- 0.0000; Class 1:AUC-ROC- 0.7544, F1 Score- 0.2105; Class 2:AUC-ROC- 0.5063, F1 Score- 0.0000; Class 3:AUC-ROC- 0.6449, F1 Score- 0.0099; Macro-Average AUC-ROC: 0.6477,Macro-Average F1 Score: 0.0551
Validation Loss: 1.3834, Validation Accuracy: 0.2500, Validation Edge Accura

In [9]:
test_smote(data, encoder, classifier, decoder, test_idx, adj_old = None, mode = 'nsm', dataset = "Test")

Test Loss: 1.0463, Test Accuracy: 0.9455, Test Edge Accuracy: 1.0000
Class 0:AUC-ROC- 0.9823, F1 Score- 0.7768; Class 1:AUC-ROC- 0.9878, F1 Score- 0.3507; Class 2:AUC-ROC- 0.9579, F1 Score- 0.3127; Class 3:AUC-ROC- 0.9811, F1 Score- 0.3957; Macro-Average AUC-ROC: 0.9773,Macro-Average F1 Score: 0.4590


(0.9454545454545454, 0.9772640190645052, 0.45896981007617116)